In [ ]:
#! pip install geopandas
#! pip uninstall astrovision
! pip install astrovision

In [ ]:
import s3fs
import re
import numpy as np
from astrovision.plot.plot_utils import make_mosaic

import geopandas as gpd
import pandas as pd
import os

from pyproj import Transformer
from shapely.geometry import Polygon

from astrovision.data import SatelliteImage, SegmentationLabeledSatelliteImage

import rasterio
from rasterio.features import rasterize

import matplotlib.pyplot as plt

## Test API

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr

In [ ]:
%env AWS_ACCESS_KEY_ID=projet-slums-key
%env AWS_SECRET_ACCESS_KEY=
del os.environ["AWS_SESSION_TOKEN"]

In [ ]:
import requests

image_path = "projet-slums-detection/golden-test/patchs/segmentation/PLEIADES/MAYOTTE_CLEAN/2022/250/ORT_976_2022_0524_8587_U38S_8Bits_0011.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1797_U20N_8bits_0037.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1799_U20N_8bits_0049.jp2"
# image_path = "projet-slums-detection/data-preprocessed/patchs/BDTOPO/segmentation/PLEIADES/GUADELOUPE/2022/250/train/ORT_971_2022_0627_1801_U20N_8bits_0042.jp2"

response = requests.get(
    "https://satellite-images-inference.lab.sspcloud.fr/predict", params={"image": image_path}
)

lsi = SegmentationLabeledSatelliteImage(
    SatelliteImage.from_raster(
        file_path=f"/vsis3/{image_path}",
        dep=None,
        date=None,
        n_bands=3,
    ),
    np.array(response.json()["mask"]),
)
plot = lsi.plot(bands_indices=[0, 1, 2])

# lsi.label

## Test make_mosaic

In [ ]:
si = SatelliteImage.from_raster(
    file_path=f"/vsis3/{image_path}",
    dep=None,
    date=None,
    n_bands=3,
)

liste_si = si.split(250)
si2 = make_mosaic(liste_si, [0, 1, 2])
si2.array = si2.array.astype(int)
si2.plot([0, 1, 2])

### Chargement des îlots

Ya plus les IOU ?
stocvkage moyenne variances dans les artifact MLFlow
Ici je vais charger une table de contour géométriique d'ilot et créer une fonction qui à partir d'un contour polygonal d'îlot  et d'un modèle :

1) récupère les images s'intersectant avec l'îlot (une année donnée) via leur nom (cooronnées du coin incluse dans le nom)
2) un modèle vient faire des prédictions 01 ou en proba sur les images récupérées
3) On somme les probabilités ou les 1 contenus dans l'ilôt (partie la plus subtile ? peut on cropper un raster ? je l'espère)
4) on peut donc attribuer à l'îlot uner surface batie prédite par le modèle
5) se permettre une représentation cartographique en représentant sur un même graphe le fond de carte donné par les patchs qui entour e complètement l'ilot (zone rectangulaire), les prédictions du modèle, et lîlot en question (ou bien d'une prédiction de réseau de neurones en probabilités) somme les1 du masque dans le contour de l'ilot

Je travaille à partir du jeu de test d'images de Mayotte dans un premeir temps ppour avoir des masques prédits hors modèle et des ilots martiniques

Petites remarques comme ça : Quand on est en prod on veut toutes les images du coup dans l'idée plutôt chopper les grosses images qui s'intersectent dans l'ilot, les spliter faire les prédictions et ne garder que les points dans l'ilot : moralité On repart des grosses images m^me si l'intersection est moins fine 
Problème d'EPSG

PB : l'installation de requirements marche pas

 - ok dans  un premier temps je vais prendre l'le nom de l'image  récupérer les coordonnées
 - les transformer en point
 - et trouver quel ilot intersecte l'image


mc cp --recursive  s3/projet-slums-detection/ilots/ ilots

In [ ]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://" + "minio.lab.sspcloud.fr"})
fs.download(rpath="projet-slums-detection/ilots", lpath="ilots", recursive=True)

In [ ]:
# Load a shapefile
fp = "ilots/ilots_976.shp"
ilots_mayotte = gpd.read_file(fp)
ilots_mayotte.head()

Je vais sélectionner les images qui tombent dans un ilot (ie où il y a intersection) et voir si je ne peux pas mettre 0 partou à l'extérieur de l'ilot

In [ ]:
bucket_name = "projet-slums-detection"
directory_path = "data-raw/PLEIADES/MAYOTTE/2022/"

full_path = f"{bucket_name}/{directory_path}"
files = fs.ls(full_path)

print(files[:6])

In [ ]:
# Function to extract coordinates from filename, adjusted for meters
def extract_coordinates(filename):
    matches = re.findall(r"\d{4}", filename)
    if len(matches) >= 2:
        return int(matches[-2]) * 1000, int(matches[-1]) * 1000
    return None, None


# Initialize the transformer from EPSG:4471 to WGS84
transformer = Transformer.from_crs("EPSG:4471", "EPSG:4326")

# Define the length of the square's side (in meters)
side_length = 1000  # Adjust as needed

# Process filenames and transform coordinates to create squares
data = []
for filename in files:
    x, y = extract_coordinates(filename)

    if x and y:
        # Define the square's corners in EPSG:4471
        top_left = (x, y)
        top_right = (x + side_length, y)
        bottom_right = (x + side_length, y - side_length)
        bottom_left = (x, y - side_length)

        # Create a polygon from these corners
        square = Polygon([top_left, top_right, bottom_right, bottom_left])

        # Transform each corner to latitude and longitude
        transformed_corners = [
            (lat, lon)
            for lon, lat in [transformer.transform(*corner) for corner in square.exterior.coords]
        ]

        # Create a new polygon with the transformed coordinates
        square_lat_lon = Polygon(transformed_corners)

        # Store the filename and the polygon
        data.append({"filename": filename, "geometry": square_lat_lon})

# Create a DataFrame
df = pd.DataFrame(data)

filename_to_polygon = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
filename_to_polygon.crs = "EPSG:4326"
# This DataFrame now contains the filenames and the corresponding square polygons in latitude and longitude
filename_to_polygon.head()

In [ ]:
intersection_ilot_filename = gpd.sjoin(
    ilots_mayotte, filename_to_polygon, how="inner", predicate="intersects"
)

intersection_ilot_filename.groupby("code_ilot").size().reset_index(name="count").sort_values(
    by="count", ascending=False
).to_csv("comptage_ilots.csv", index=False)

## Traitement d'un îlot :

- A un ilot correspond une liste de grosses images.
- On split chacune de ces grosses images en 64 images, on applique le modèle dessus et on refusionne l'ensemble (images et masques)
- ensuite je détermine le masque ilot qui donne l'apaprtenance de chaque pixel a l'ilot sur la grosse image ce qui me permet d'extirper le masque ilot


Je commence avec l'ilot 824 et ses 3 images

In [ ]:
num_ilot = "0136"
ilots_mayotte.loc[ilots_mayotte.code_ilot == num_ilot].geometry.iloc[0]

In [ ]:
filenames_ilot = intersection_ilot_filename.loc[
    intersection_ilot_filename["code_ilot"] == num_ilot
].filename

polygons_ilot = intersection_ilot_filename.loc[
    intersection_ilot_filename["code_ilot"] == num_ilot
].geometry

In [ ]:
def construire_raster_ilot(polygon, filename):
    # je recupere le carré de la grosse image sous forme de polygone
    polygon_image = filename_to_polygon.loc[
        filename_to_polygon["filename"] == filename
    ].geometry.iloc[0]

    # intersection avec l'ilot en cours
    intersection_ilot_image_polygon = polygon_image.intersection(polygon)

    # pour visualiser seulement :
    # multipolygone_union = MultiPolygon([polygon_image, intersection_ilot_image_polygon])
    # multipolygone_union

    # rasterio.transform.from_bounds
    x_min, y_min, x_max, y_max = polygon_image.bounds
    transform = rasterio.transform.from_bounds(x_min, y_min, x_max, y_max, 2000, 2000)

    valeur_polygone = 1

    # Rasteriser le polygone actuel avec une valeur unique
    raster_polygone = rasterize(
        [(intersection_ilot_image_polygon, valeur_polygone)],
        out_shape=(2000, 2000),
        transform=transform,
    )

    return raster_polygone


def representer_bout_ilot(polygon, filename):
    raster_image = construire_raster_ilot(polygon, filename)

    si = SatelliteImage.from_raster(
        file_path="/vsis3/" + filename,
        dep=None,
        date=None,
        n_bands=3,
    )
    bands_indices = [0, 1, 2]
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(np.transpose(si.array, (1, 2, 0))[:, :, bands_indices])

    plt.xticks([])
    plt.yticks([])

    ax.imshow(raster_image, cmap="jet", alpha=0.5)
    plt.show()

In [ ]:
i = 1
polygon = polygons_ilot.iloc[i]
filename = filenames_ilot.iloc[i]

representer_bout_ilot(polygon, filename)

Objectif : superposer la mosaique d'imlage extraite et le cntour d'ilot
Superposer les prédictions du modèle
comment cropper un raster avec un polygone ?

#filename_ilot , créer la liste de si et faire la mosaique avec astrovision